In [162]:
from pytqs import tqs
import pandas as pd
import numpy as np

from datetime import date, timedelta
import datetime; 

from sklearn import neighbors
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report

import bytedtqs

In [107]:
appId = "eL38NWjpAMXlC0ifCoo71wk27zE6CS9YQismEcwVH5oNKbJf"
appKey = "VeeOKtXLxjGWE2OyIGkuqWRT33isyxtHH1xlxakeFzId6SMB"
user_id = 'wuweiwei.1012'

## 获取原始数据

In [108]:
sql_txt = """ 

set tqs.query.result.max.lines = 1000000000;

select * from dm_eo_test.team_user_info limit 1000000000
"""

In [109]:
client = tqs.TQSClient(appId=appId, appKey=appKey)
job = client.execute_query(username = user_id , sql = sql_txt)
data = pd.read_csv(job['resultUrl'])

[2021-03-05 00:57:23,174] - [INFO] - tqs_task_id: 213742950, engine: Hive, status: Created, progress : Submitting
[2021-03-05 00:57:25,197] - [INFO] - tqs_task_id: 213742950, execute id: tqs_213742950-1614877043712
[2021-03-05 00:57:25,197] - [INFO] - application tracking url: http://presto-history.byted.org/proxy/20210304_165723_84179_default
[2021-03-05 00:57:25,198] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 00:57:27,219] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 00:57:29,248] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 00:57:31,267] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 00:57:33,297] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 00:57:35,344] - [INFO] - tqs_task_id: 213742950, engine: 

[2021-03-05 01:01:40,059] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 01:01:45,087] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 01:01:50,117] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 01:01:55,145] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 01:02:00,180] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 01:02:05,212] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 01:02:10,232] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 01:02:15,266] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 01:02:20,289] - [INFO] - tqs_task_id: 213742950, eng

[2021-03-05 01:07:36,915] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 01:07:41,935] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 01:07:46,955] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 01:07:51,981] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Processing, progress : Accepted
[2021-03-05 01:07:57,042] - [INFO] - tqs_task_id: 213742950, engine: Presto, status: Completed, progress : Accepted


In [111]:
data.head()

,user_id,study_dt,active_day_gap,first_milestone_name,occupation,vip_lfc_video,vip_lfc_audio,study_flag,first_team_user,user_group,study_duration,is_clock,study_duration_pre21d,study_duration_post21d,clock_pre21d,clock_post21d
0,2422855878,2020-11-04,649,NaN,office_worker,NaN,NaN,low,0,Control,0,0,0,23329,0,10
1,2422855878,2020-11-05,650,NaN,office_worker,NaN,NaN,low,0,Control,0,0,0,23329,0,10
2,2422855878,2020-11-06,651,NaN,office_worker,NaN,1.0,low,0,Control,0,0,0,23329,0,10
3,2422855878,2020-11-07,652,NaN,office_worker,NaN,2.0,low,0,Control,3129,1,3129,23329,1,10
4,2422855878,2020-11-08,653,NaN,office_worker,NaN,3.0,low,0,Control,1379,1,4508,20200,2,9


In [112]:
data["study_duration_pre21d_new"] = data["study_duration_pre21d"] - data["study_duration"]
data["clock_pre21d_new"] = data["clock_pre21d"] - data["is_clock"]
data["first_milestone_name"] = data["first_milestone_name"].fillna("未定级")
data["vip_lfc_video"] = data["vip_lfc_video"].fillna(-1)
data["vip_lfc_audio"] = data["vip_lfc_audio"].fillna(-1)

data.head()

,user_id,study_dt,active_day_gap,first_milestone_name,occupation,vip_lfc_video,vip_lfc_audio,study_flag,first_team_user,user_group,study_duration,is_clock,study_duration_pre21d,study_duration_post21d,clock_pre21d,clock_post21d,study_duration_pre21d_new,clock_pre21d_new
0,2422855878,2020-11-04,649,未定级,office_worker,-1.0,-1.0,low,0,Control,0,0,0,23329,0,10,0,0
1,2422855878,2020-11-05,650,未定级,office_worker,-1.0,-1.0,low,0,Control,0,0,0,23329,0,10,0,0
2,2422855878,2020-11-06,651,未定级,office_worker,-1.0,1.0,low,0,Control,0,0,0,23329,0,10,0,0
3,2422855878,2020-11-07,652,未定级,office_worker,-1.0,2.0,low,0,Control,3129,1,3129,23329,1,10,0,0
4,2422855878,2020-11-08,653,未定级,office_worker,-1.0,3.0,low,0,Control,1379,1,4508,20200,2,9,3129,1


In [113]:
len(data)

44088490

## 匹配Test & Control

### 分离Data

In [114]:
test_data = data[data.user_group == 'Test']
control_data = data[(data.user_group == 'Control') & (data.first_team_user == 0)]
len(test_data)

14759

In [115]:
test_data.head()

,user_id,study_dt,active_day_gap,first_milestone_name,occupation,vip_lfc_video,vip_lfc_audio,study_flag,first_team_user,user_group,study_duration,is_clock,study_duration_pre21d,study_duration_post21d,clock_pre21d,clock_post21d,study_duration_pre21d_new,clock_pre21d_new
18153,51332425384,2021-02-04,854,未定级,unknown,-1.0,95.0,low,1,Test,1201,1,1657,12072,1,8,456,0
22546,76116283924,2021-02-09,831,未定级,office_worker,-1.0,456.0,low,1,Test,648,1,648,648,1,1,0,0
38890,51924170711,2020-12-10,715,未定级,unknown,-1.0,251.0,low,1,Test,1450,1,4118,2985,2,2,2668,1
50101,96395326692,2021-01-11,833,未定级,unknown,-1.0,70.0,silence,1,Test,150,0,150,4865,0,3,0,0
65886,98965454454,2021-02-22,855,未定级,office_worker,-1.0,203.0,medium,1,Test,2054,1,36529,18556,14,8,34475,13


In [165]:
match_data = pd.DataFrame(columns = ["study_flag", "user_id_t","clock_post21d_t", "clock_post21d_c", \
                                     "study_duration_post21d_t", "study_duration_post21d_c"])

#### KNN match

In [166]:
test_num = len(test_data)
match_var = ["vip_lfc_video", "vip_lfc_audio", "study_duration_pre21d_new", "clock_pre21d_new"]
metric_var = ["clock_post21d", "study_duration_post21d"]

In [173]:
for index in range(501, test_num):
    # test user info
    test_data_user = test_data.iloc[index]
    user_id_t = test_data_user.user_id
    study_duration_post21d_t = test_data_user.study_duration_post21d
    clock_post21d_t = test_data_user.clock_post21d
    study_flag = test_data_user.study_flag
    
    join_dt = test_data_user.study_dt
    first_milestone_name = test_data_user.first_milestone_name
    occupation = test_data_user.occupation
    
    
    test_x = test_data_user[match_var]

    # control pool & well match
    control_pool = control_data[(control_data.study_dt == join_dt) & \
                                (control_data.first_milestone_name == first_milestone_name) & \
                                (control_data.occupation == occupation)]

    control_x = control_pool[match_var]
    control_y = control_pool[metric_var]

    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(control_x.to_numpy())

    test_x_list = []  
    test_x_list.append(test_x.to_numpy())
    distances, indices = nbrs.kneighbors(test_x_list)

    control_index = indices[0][0]

    study_duration_post21d_c = control_pool.study_duration_post21d.iloc[control_index]
    clock_post21d_c = control_pool.clock_post21d.iloc[control_index]

    # record paired tc
    match_data = match_data.append({"study_flag": study_flag, "user_id_t": user_id_t, \
                                    "clock_post21d_t": clock_post21d_t, "clock_post21d_c": clock_post21d_c,\
                                    "study_duration_post21d_t": study_duration_post21d_t, \
                                    "study_duration_post21d_c": study_duration_post21d_c}, \
                                   ignore_index = True)
    print('------------' + str(index) + ' ended at ' + str(datetime.datetime.now()) + '------------')
    match_data.to_csv(r'./match_data.csv', encoding='utf_8_sig')

------------501 ended at 2021-03-05 10:45:30.278899------------
------------502 ended at 2021-03-05 10:45:39.141493------------
------------503 ended at 2021-03-05 10:45:48.089595------------
------------504 ended at 2021-03-05 10:45:58.264434------------
------------505 ended at 2021-03-05 10:46:07.958113------------
------------506 ended at 2021-03-05 10:46:16.943026------------
------------507 ended at 2021-03-05 10:46:26.706649------------
------------508 ended at 2021-03-05 10:46:35.506677------------
------------509 ended at 2021-03-05 10:46:44.483272------------
------------510 ended at 2021-03-05 10:46:54.442390------------
------------511 ended at 2021-03-05 10:47:10.558836------------
------------512 ended at 2021-03-05 10:47:19.167428------------
------------513 ended at 2021-03-05 10:47:27.858706------------
------------514 ended at 2021-03-05 10:47:43.905675------------
------------515 ended at 2021-03-05 10:47:54.012657------------
------------516 ended at 2021-03-05 10:4

------------630 ended at 2021-03-05 11:05:51.195135------------
------------631 ended at 2021-03-05 11:05:59.993186------------
------------632 ended at 2021-03-05 11:06:08.602088------------
------------633 ended at 2021-03-05 11:06:23.883067------------
------------634 ended at 2021-03-05 11:06:32.339315------------
------------635 ended at 2021-03-05 11:06:41.693618------------
------------636 ended at 2021-03-05 11:06:51.458397------------
------------637 ended at 2021-03-05 11:06:59.771905------------
------------638 ended at 2021-03-05 11:07:09.526104------------
------------639 ended at 2021-03-05 11:07:17.888859------------
------------640 ended at 2021-03-05 11:07:25.905854------------
------------641 ended at 2021-03-05 11:07:35.731447------------
------------642 ended at 2021-03-05 11:07:54.960196------------
------------643 ended at 2021-03-05 11:08:03.338784------------
------------644 ended at 2021-03-05 11:08:12.579412------------
------------645 ended at 2021-03-05 11:0

------------759 ended at 2021-03-05 11:27:25.311383------------
------------760 ended at 2021-03-05 11:27:44.725252------------
------------761 ended at 2021-03-05 11:28:02.563507------------
------------762 ended at 2021-03-05 11:28:12.382801------------
------------763 ended at 2021-03-05 11:28:20.973049------------
------------764 ended at 2021-03-05 11:28:30.909603------------
------------765 ended at 2021-03-05 11:28:40.732640------------
------------766 ended at 2021-03-05 11:28:50.491487------------
------------767 ended at 2021-03-05 11:28:59.015105------------
------------768 ended at 2021-03-05 11:29:07.790920------------
------------769 ended at 2021-03-05 11:29:23.611294------------
------------770 ended at 2021-03-05 11:29:41.338739------------
------------771 ended at 2021-03-05 11:29:55.575171------------
------------772 ended at 2021-03-05 11:30:03.921061------------
------------773 ended at 2021-03-05 11:30:12.539754------------
------------774 ended at 2021-03-05 11:3

------------888 ended at 2021-03-05 11:49:56.324510------------
------------889 ended at 2021-03-05 11:50:04.921193------------
------------890 ended at 2021-03-05 11:50:13.628204------------
------------891 ended at 2021-03-05 11:50:22.305821------------
------------892 ended at 2021-03-05 11:50:30.976955------------
------------893 ended at 2021-03-05 11:50:40.949907------------
------------894 ended at 2021-03-05 11:50:49.588788------------
------------895 ended at 2021-03-05 11:50:57.890752------------
------------896 ended at 2021-03-05 11:51:06.578622------------
------------897 ended at 2021-03-05 11:51:15.245347------------
------------898 ended at 2021-03-05 11:51:23.543791------------
------------899 ended at 2021-03-05 11:51:32.298429------------
------------900 ended at 2021-03-05 11:51:41.088594------------
------------901 ended at 2021-03-05 11:51:56.557445------------
------------902 ended at 2021-03-05 11:52:05.543127------------
------------903 ended at 2021-03-05 11:5

------------1016 ended at 2021-03-05 12:10:51.005188------------
------------1017 ended at 2021-03-05 12:11:08.485247------------
------------1018 ended at 2021-03-05 12:11:17.380537------------
------------1019 ended at 2021-03-05 12:11:26.228030------------
------------1020 ended at 2021-03-05 12:11:34.785537------------
------------1021 ended at 2021-03-05 12:11:43.391871------------
------------1022 ended at 2021-03-05 12:11:51.731727------------
------------1023 ended at 2021-03-05 12:12:00.642866------------
------------1024 ended at 2021-03-05 12:12:09.542855------------
------------1025 ended at 2021-03-05 12:12:18.398006------------
------------1026 ended at 2021-03-05 12:12:26.696754------------
------------1027 ended at 2021-03-05 12:12:35.273741------------
------------1028 ended at 2021-03-05 12:12:43.768174------------
------------1029 ended at 2021-03-05 12:12:52.550568------------
------------1030 ended at 2021-03-05 12:13:01.206468------------
------------1031 ended at

...

------------3859 ended at 2021-03-05 19:58:22.068208------------
------------3860 ended at 2021-03-05 19:58:30.804441------------
------------3861 ended at 2021-03-05 19:58:39.128829------------
------------3862 ended at 2021-03-05 19:58:47.505840------------
------------3863 ended at 2021-03-05 19:58:56.263604------------
------------3864 ended at 2021-03-05 19:59:04.499283------------
------------3865 ended at 2021-03-05 19:59:22.205705------------
------------3866 ended at 2021-03-05 19:59:31.813888------------
------------3867 ended at 2021-03-05 19:59:40.422224------------
------------3868 ended at 2021-03-05 19:59:48.950495------------
------------3869 ended at 2021-03-05 19:59:57.730012------------
------------3870 ended at 2021-03-05 20:00:06.647417------------
------------3871 ended at 2021-03-05 20:00:15.528812------------
------------3872 ended at 2021-03-05 20:00:24.252416------------
------------3873 ended at 2021-03-05 20:00:33.156609------------
------------3874 ended at

------------3995 ended at 2021-03-05 20:20:39.046132------------
------------3996 ended at 2021-03-05 20:20:47.571766------------
------------3997 ended at 2021-03-05 20:20:56.072252------------
------------3998 ended at 2021-03-05 20:21:04.839020------------
------------3999 ended at 2021-03-05 20:21:13.723191------------
------------4000 ended at 2021-03-05 20:21:22.638883------------
------------4001 ended at 2021-03-05 20:21:31.449101------------
------------4002 ended at 2021-03-05 20:21:39.725945------------
------------4003 ended at 2021-03-05 20:21:48.187490------------
------------4004 ended at 2021-03-05 20:21:56.960823------------
------------4005 ended at 2021-03-05 20:22:05.816541------------
------------4006 ended at 2021-03-05 20:22:14.842436------------
------------4007 ended at 2021-03-05 20:22:23.760476------------
------------4008 ended at 2021-03-05 20:22:32.296131------------
------------4009 ended at 2021-03-05 20:22:41.253889------------
------------4010 ended at

------------4122 ended at 2021-03-05 20:40:47.303762------------
------------4123 ended at 2021-03-05 20:40:56.101575------------
------------4124 ended at 2021-03-05 20:41:04.628478------------
------------4125 ended at 2021-03-05 20:41:13.156390------------
------------4126 ended at 2021-03-05 20:41:28.744501------------
------------4127 ended at 2021-03-05 20:41:37.418516------------
------------4128 ended at 2021-03-05 20:41:54.416747------------
------------4129 ended at 2021-03-05 20:42:11.138464------------
------------4130 ended at 2021-03-05 20:42:19.826983------------
------------4131 ended at 2021-03-05 20:42:28.480705------------
------------4132 ended at 2021-03-05 20:42:37.263050------------
------------4133 ended at 2021-03-05 20:42:46.013652------------
------------4134 ended at 2021-03-05 20:42:54.354398------------
------------4135 ended at 2021-03-05 20:43:02.642391------------
------------4136 ended at 2021-03-05 20:43:11.503874------------
------------4137 ended at

------------4258 ended at 2021-03-05 21:03:51.360315------------
------------4259 ended at 2021-03-05 21:04:00.193129------------
------------4260 ended at 2021-03-05 21:04:08.514208------------
------------4261 ended at 2021-03-05 21:04:16.839170------------
------------4262 ended at 2021-03-05 21:04:25.770426------------
------------4263 ended at 2021-03-05 21:05:00.316806------------
------------4264 ended at 2021-03-05 21:05:08.911180------------
------------4265 ended at 2021-03-05 21:05:17.425725------------
------------4266 ended at 2021-03-05 21:05:26.004704------------
------------4267 ended at 2021-03-05 21:05:40.932161------------
------------4268 ended at 2021-03-05 21:05:49.076292------------
------------4269 ended at 2021-03-05 21:05:57.872730------------
------------4270 ended at 2021-03-05 21:06:06.459394------------
------------4271 ended at 2021-03-05 21:06:15.240120------------
------------4272 ended at 2021-03-05 21:06:23.860186------------
------------4273 ended at

ValueError: Found array with 0 sample(s) (shape=(0, 4)) while a minimum of 1 is required.

In [170]:
range(500, 14759)

range(500, 14759)